In [1]:
import torch
from transformers import (
    AutoModelForCausalLM,
    LlamaTokenizerFast,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig,
)
from peft import (
    LoraConfig,
    get_peft_model,
)

2024-04-17 23:36:21.332751: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-17 23:36:21.438877: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-17 23:36:21.917713: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2024-04-17 23:36:21.917767: W tensorflow/compiler/xla/stream_exe

In [2]:
# https://huggingface.co/meta-llama/Llama-2-7b-hf
# base_model = "meta-llama/Llama-2-7b-hf"
base_model = "TinyPixel/Llama-2-7B-bf16-sharded"

In [3]:
# # Fine-tuned model
# new_model = "llama-2-7b-hf-fine-tuned-test1"

In [3]:
# QLoRA 모델을 사용하기 위한 설정

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16 
)



In [4]:
# 라마2 모델 불러오기
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=bnb_config,
    device_map="auto",
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

In [5]:
# 토크나이저 로드
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false" # 토크나이저 병렬처리 방지(오류 방지)
os.environ['TRANSFORMERS_NO_ADVISORY_WARNINGS'] = 'true' # __cell__ 오류 방지

tokenizer = LlamaTokenizerFast.from_pretrained(
    base_model,
    trust_remote_code=True
)
tokenizer.pad_token = tokenizer.eos_token # 패딩 토큰을 문장의 끝으로 설정 </s>
tokenizer.padding_side = "right" # 패딩을 문장 뒤에 추가

tokenizer_config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

In [6]:
# 학습 양식
import json
from datasets import load_dataset
file_name = 'preprocess/dataset3/ratings.json'

In [7]:
import json

# JSON 파일 불러오기
input_file = file_name
with open(input_file, 'r') as f:
    data = json.load(f)

# 불러온 데이터 확인
# print(data)

In [8]:
# print(data[1])

print(data[0]["ratings"]) # 영화 평점들 정보 모음
print(type(data[0]["ratings"])) 

print(data[0]["ratings"][0]) # 평점 1개
print(type(data[0]["ratings"][0])) 
print(data[0]["order"])

[{'imdbId': 'tt114709', 'rating': 4, 'title': 'Toy Story (1995)', 'genres': 'Adventure|Animation|Children|Comedy|Fantasy', 'timestamp': 964982703}, {'imdbId': 'tt113228', 'rating': 4, 'title': 'Grumpier Old Men (1995)', 'genres': 'Comedy|Romance', 'timestamp': 964981247}, {'imdbId': 'tt113277', 'rating': 4, 'title': 'Heat (1995)', 'genres': 'Action|Crime|Thriller', 'timestamp': 964982224}, {'imdbId': 'tt114369', 'rating': 5, 'title': 'Seven (a.k.a. Se7en) (1995)', 'genres': 'Mystery|Thriller', 'timestamp': 964983815}, {'imdbId': 'tt114814', 'rating': 5, 'title': 'Usual Suspects, The (1995)', 'genres': 'Crime|Mystery|Thriller', 'timestamp': 964982931}, {'imdbId': 'tt116367', 'rating': 3, 'title': 'From Dusk Till Dawn (1996)', 'genres': 'Action|Comedy|Horror|Thriller', 'timestamp': 964982400}, {'imdbId': 'tt115734', 'rating': 5, 'title': 'Bottle Rocket (1996)', 'genres': 'Adventure|Comedy|Crime|Romance', 'timestamp': 964980868}, {'imdbId': 'tt112573', 'rating': 4, 'title': 'Braveheart (1

In [9]:
instruction = '''
사용자의 영화 평점 기록을 바탕으로 영화를 추천해줍니다. 영화 title 리스트를 생성해주고, 관련한 메타메이터를 보여줍니다. must return json format
'''

In [10]:
# 데이터 매핑 함수 정의
def map_data_to_format(example):

    ratings = json.dumps(example['ratings'], ensure_ascii=False)
    order_list = json.dumps(example["order"], ensure_ascii=False)
    
    text = (
        f"user_ratings_info:\n{ratings}\n\n"
        # f"keyword_info:\n{genres_str}\n\n"
        # f"overview_tk_str:\n{overview_tk_str}\n\n"
        # f"tagline_tk_str:\n{tagline_tk_str}\n\n"
    )
    
    # completion은 모델이 생성해야 할 예상 출력을 포함합니다.

    # 여기서는 추천 목록 영화 리스트 imdb_id를 반환합니다.
    completion = f"{{\"order_list\": {order_list} \
    }}"
    
    return {'text': text, 'completion': completion}

In [11]:
# 데이터셋 로드
data = load_dataset('json', data_files=file_name, split="train")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
print(data)

Dataset({
    features: ['order', 'ratings', 'userId'],
    num_rows: 10084
})


In [13]:
# 데이터 매핑 적용
mapped_data = data.map(map_data_to_format)

Map:   0%|          | 0/10084 [00:00<?, ? examples/s]

In [15]:
#print(mapped_data['overview_tk'])
# print(mapped_data['tagline_tk'])
# print(mapped_data['movie_info'])
# print(mapped_data['text'])
# print(mapped_data['completion'])



In [16]:
print(mapped_data)


Dataset({
    features: ['order', 'ratings', 'userId', 'text', 'completion'],
    num_rows: 10084
})


In [17]:
# 데이터셋 분할
split_data = mapped_data.train_test_split(test_size=0.1)  # 10%를 테스트셋으로 사용

train_set = split_data['train']
eval_set = split_data['test']

train_set = train_set.map(lambda samples: tokenizer(samples["text"], padding=True, truncation=True, return_tensors="pt"), batched=True)
eval_set = eval_set.map(lambda samples: tokenizer(samples["text"], padding=True, truncation=True, return_tensors="pt"), batched=True)

# lora 파라미터 설정
peft_params = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)


Map:   0%|          | 0/9075 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1009 [00:00<?, ? examples/s]

In [18]:
print(train_set[3]["text"])
print(train_set[20]["text"])
print(train_set[120]["text"])

user_ratings_info:
[{"genres": "Action|Drama|War", "imdbId": "tt120863", "rating": 4, "timestamp": 959896591, "title": "Thin Red Line, The (1998)"}, {"genres": "Drama|Mystery|Thriller", "imdbId": "tt134273", "rating": 2, "timestamp": 959896430, "title": "8MM (1999)"}, {"genres": "Action|Sci-Fi|Thriller|Western", "imdbId": "tt70909", "rating": 4, "timestamp": 959896237, "title": "Westworld (1973)"}, {"genres": "Action|Sci-Fi|Thriller", "imdbId": "tt133093", "rating": 4, "timestamp": 959896015, "title": "Matrix, The (1999)"}, {"genres": "Action|Adventure|Sci-Fi", "imdbId": "tt120915", "rating": 2, "timestamp": 959895496, "title": "Star Wars: Episode I - The Phantom Menace (1999)"}, {"genres": "Crime|Drama|Mystery|Thriller", "imdbId": "tt144214", "rating": 3, "timestamp": 959896430, "title": "General's Daughter, The (1999)"}, {"genres": "Thriller", "imdbId": "tt137363", "rating": 4, "timestamp": 959896305, "title": "Arlington Road (1999)"}, {"genres": "Drama|Horror|Mystery", "imdbId": "tt

In [19]:
print(train_set[3]["completion"])
print(train_set[20]["completion"])
print(train_set[120]["completion"])

{"order_list": ["tt167404", "tt120863", "tt137363", "tt70909", "tt155267", "tt133093", "tt73802", "tt144214", "tt134273", "tt120915"]     }
{"order_list": ["tt119654", "tt87469", "tt78346", "tt100802", "tt82198", "tt120611", "tt119094", "tt119116", "tt104437", "tt129387"]     }
{"order_list": ["tt430922", "tt830515", "tt870111", "tt1013753", "tt1205489", "tt1125849", "tt959337", "tt421715", "tt985699", "tt1099212"]     }


In [21]:
model = get_peft_model(model, peft_params)

# prameter
epochs = 3 # 10

batch_size = 2

lr = 2e-4

training_params = TrainingArguments(
    output_dir="models",
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=16,
    optim="adamw_torch",
    save_strategy="epoch",
    evaluation_strategy="steps",
    logging_strategy="steps",
    eval_steps=20,
    logging_steps=20,
    learning_rate=lr,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    dataloader_num_workers=1,
)


In [22]:
# ! wandb login --relogin


In [23]:
import transformers
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

trainer = Trainer(
    model=model,
    args=training_params,
    train_dataset=train_set,
    eval_dataset=eval_set,
    tokenizer=tokenizer,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

trainer.train()

/home/gpuadmin/anaconda3/envs/llm_test/lib/python3.8/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


  0%|          | 0/849 [00:00<?, ?it/s]

In [ ]:
from tensorboard import notebook
log_dir = "./models" 
notebook.start("--logdir {} --port 4000".format(log_dir))

In [35]:
instruction = '''
사용자가 입력한 키워드를 통해 영화를 추천해줍니다. 영화 title 리스트를 생성해주고, 관련한 메타메이터를 보여줍니다. must return json format
'''
key_info_str = {"keywords": ['toy', 'love', 'universe']}
# genres_str={"genres":['animation']}


In [36]:
# ###instruction:
# {instruction}

text = f'''

keyword_info:
{key_info_str}


'''
inputs = tokenizer(text, return_tensors="pt").to("cuda")
outputs = model.generate(
    input_ids=inputs["input_ids"].to("cuda"), 
    attention_mask=inputs["attention_mask"], 
    max_new_tokens=256,
    early_stopping=True,
    pad_token_id=tokenizer.eos_token_id
)
output = tokenizer.decode(outputs[0])
print(output)


<s> 

keyword_info:
{'keywords': ['toy', 'love', 'universe']}









































































































































































!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


wandb: Network error (ReadTimeout), entering retry loop.
